# Laboratorio 8 - Deep Learning y sistemas inteligentes
### Repo: https://github.com/SebasJuarez/StoreItemDemand
### Sebastian Juárez - 21471
### Javier Prado - 21486
### Bryan España - 21550

### Preparación

In [ ]:
import os, math, warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
warnings.filterwarnings("ignore")


SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

DATA_DIR = "./Data"
TRAIN_PATH = os.path.join(DATA_DIR, "train.csv")
TEST_PATH  = os.path.join(DATA_DIR, "test.csv")

HORIZON_DAYS = 90
WINDOW_SIZE  = 180
MIN_SERIES_LEN = WINDOW_SIZE + HORIZON_DAYS + 1


### Preparación de datos

#### Limpieza de celdas

In [ ]:
train = pd.read_csv(TRAIN_PATH)
test  = pd.read_csv(TEST_PATH)

train["date"] = pd.to_datetime(train["date"])
train = train.sort_values(["store","item","date"]).reset_index(drop=True)


print(train.head())
print(train.isna().sum())


if train["sales"].isna().any():
    train["sales"] = train["sales"].fillna(0)

def winsorize_series(s, k=1.5):
    q1, q3 = s.quantile(0.25), s.quantile(0.75)
    iqr = q3 - q1
    lo, hi = q1 - k*iqr, q3 + k*iqr
    return s.clip(lower=lo, upper=hi)

train["sales_w"] = (
    train.groupby(["store","item"])["sales"]
         .transform(lambda s: winsorize_series(s))
)

train["sales_log"] = np.log1p(train["sales_w"])


        date  store  item  sales
0 2013-01-01      1     1     13
1 2013-01-02      1     1     11
2 2013-01-03      1     1     14
3 2013-01-04      1     1     13
4 2013-01-05      1     1     10
date     0
store    0
item     0
sales    0
dtype: int64


#### Transformaciones

In [ ]:
train["dow"]   = train["date"].dt.dayofweek
train["month"] = train["date"].dt.month

train["dow_s"]   = train["dow"]   / 6.0
train["month_s"] = (train["month"] - 1) / 11.0

feat_cols = ["sales_log", "dow_s", "month_s"]
key_cols  = ["store","item","date"]
data = train[key_cols + feat_cols].copy()
data.head()


,store,item,date,sales_log,dow_s,month_s
0,1,1,2013-01-01,2.639057,0.166667,0.0
1,1,1,2013-01-02,2.484907,0.333333,0.0
2,1,1,2013-01-03,2.708050,0.500000,0.0
3,1,1,2013-01-04,2.639057,0.666667,0.0
4,1,1,2013-01-05,2.397895,0.833333,0.0


### Preprocesamiento

In [ ]:
last_date = data["date"].max()
val_start = last_date - pd.Timedelta(days=HORIZON_DAYS-1)

print("Última fecha:", last_date.date(), "| Inicio de valid:", val_start.date())

data["split"] = np.where(data["date"] >= val_start, "valid", "train")

Última fecha: 2017-12-31 | Inicio de valid: 2017-10-03


In [ ]:
def build_sequences(df_one_series, window=WINDOW_SIZE, horizon=HORIZON_DAYS):
    feats = df_one_series[feat_cols].values
    dates = df_one_series["date"].values
    splits = df_one_series["split"].values

    X_tr, y_tr, X_va, y_va = [], [], [], []

    for start in range(0, len(df_one_series) - (window + horizon) + 1):
        end_w = start + window
        end_h = end_w + horizon

        X_win = feats[start:end_w]
        y_win = df_one_series["sales_log"].values[end_w:end_h]

        target_last_date = df_one_series["date"].iloc[end_h - 1]
        if target_last_date >= val_start:
            X_va.append(X_win)
            y_va.append(y_win)
        else:
            X_tr.append(X_win)
            y_tr.append(y_win)

    return X_tr, y_tr, X_va, y_va

X_train, y_train, X_valid, y_valid = [], [], [], []

for (st, it), g in data.groupby(["store","item"]):
    g = g.sort_values("date")
    if len(g) < MIN_SERIES_LEN:
        continue
    Xt, yt, Xv, yv = build_sequences(g)
    if Xt:
        X_train.extend(Xt)
        y_train.extend(yt)
    if Xv:
        X_valid.extend(Xv)
        y_valid.extend(yv)

X_train = np.array(X_train, dtype=np.float32)
y_train = np.array(y_train, dtype=np.float32)
X_valid = np.array(X_valid, dtype=np.float32)
y_valid = np.array(y_valid, dtype=np.float32)

X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

((733500, 180, 3), (733500, 90), (45000, 180, 3), (45000, 90))

### Modelado - LSTM

In [ ]:
n_timesteps = X_train.shape[1]
n_features  = X_train.shape[2]
n_outputs   = y_train.shape[1]

def build_lstm_model(timesteps, nfeat, horizon):
    inputs = keras.Input(shape=(timesteps, nfeat))
    x = layers.LSTM(64, return_sequences=True)(inputs)
    x = layers.Dropout(0.2)(x)
    x = layers.LSTM(32)(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(64, activation="relu")(x)
    outputs = layers.Dense(horizon, activation="linear")(x)
    model = keras.Model(inputs, outputs, name="lstm_forecaster")
    return model

model = build_lstm_model(n_timesteps, n_features, n_outputs)
model.summary()


Model: "lstm_forecaster"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 180, 3)]          0         
                                                                 
 lstm (LSTM)                 (None, 180, 64)           17408     
                                                                 
 dropout (Dropout)           (None, 180, 64)           0         
                                                                 
 lstm_1 (LSTM)               (None, 32)                12416     
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense (Dense)               (None, 64)                2112      
                                                                 
 dense_1 (Dense)             (None, 90)           